In [1]:
import numpy as np
import pandas as pd
# pd.set_option('float_format', '{:f}'.format)
import matplotlib.pyplot as plt

from sympy.combinatorics import Permutation
from sympy import init_printing
init_printing(perm_cyclic=False, pretty_print=False)
from copy import deepcopy as dc

from collections import namedtuple
from tqdm import tqdm
import time

# методы из прошлой лабы

In [2]:
def norm(x ):
    return np.sqrt(sum([xi**2 for xi in x]))

def scalar_product(x,y):
    return (sum([xi*yi for xi, yi in zip(x,y)]))

def mat_vec_product(a, x):
    assert len(a[0]) == len(x)

    return np.array([
        scalar_product(y,x)
        for y in a
    ])

def matrix_product(a, b):
    k, l = len(a), len(a[0])
    l2, m = len(b), len(b[0])
    assert l == l2

    return np.array(
        [
            [
                sum(a[i][t]*b[t][j] for t in range(l))
                for j in range(m)
            ]
            for i in range(k)
        ]
    )

def transpose(a):
    return np.array([
        [a[j][i]for j in range(len(a))]
        for i in range(len(a[0]))
    ])



In [3]:
def plotter(f, a, b, dots_num=100):
    x = np.linspace(a,b, dots_num, True)
    y = f(x)
    plt.plot(x,y)
    plt.show()

# метод гаусса

In [23]:
NORMAL = 1
SWAP_ROWS = 2
SWAP_COLS = 3
SWAP_ALL = 4

def max_elem_in_matrix(A, iter_num):
    m = 0
    mi, mj = -1,-1
    for i in range(iter_num, len(A)):
        for j in range(iter_num, len(A)):
            if abs(A[i][j]) > m:
                m = abs(A[i][j])
                mi, mj = i,j
    return mi, mj

def max_elem_in_row(A, iter_num):
    m = 0
    mi, mj = -1,-1
    i = iter_num
    for j in range(iter_num, len(A)):
        if abs(A[i][j]) > m:
            m = abs(A[i][j])
            mi, mj = i,j

    return mi, mj


def max_elem_in_column(A, iter_num):
    m = 0
    mi, mj = -1,-1
    j = iter_num
    for i in range(iter_num, len(A)):
        if abs(A[i][j]) > m:
            m = abs(A[i][j])
            mi, mj = i,j
    return mi, mj

def permute_rows(A, f, i, j):
    for t in range(len(A)):
        A[i][t], A[j][t] = A[j][t], A[i][t]
    f[i], f[j] = f[j], f[i]
    return A, f

def permute_cols(A, f, i, j):
    for t in range(len(A)):
        A[t][i], A[t][j] = A[t][j], A[t][i]
    return A, f

def gaussian(A, f, mode=NORMAL):
    assert len(A) == len(A[0]) == len(f)
    A, f = dc(A), dc(f)
    D = np.zeros((len(A), len(A)))
    Dr = np.zeros((len(A),))
    N = len(A)
    p = Permutation()

    # прямой ход
    for iter_num in range(N-1):
        for t in range(iter_num+1, N):
            c = A[t, iter_num]/A[iter_num, iter_num]
            A[t] = A[t] - c*A[iter_num]
            D[t] = abs(c) * abs(A[iter_num]) + abs(A[t])
            f[t] = f[t] - c*f[iter_num]
            Dr[t] = abs(f[t]) + abs(c) + abs(f[iter_num])

    # обратный ход
    x = np.zeros(N)
    for iter_num in range(N-1, -1, -1):
        x[iter_num] = (f[iter_num] - scalar_product(x, A[iter_num])) / A[iter_num, iter_num]

    # перестановка элементов x
    for i,j in p.transpositions():
        i,j = i-1, j-1
        x[i], x[j] = x[j], x[i]

    print(D)
    print(Dr)
    print('delta norm:', max(np.linalg.norm(D), np.linalg.norm(Dr)))


    return x

## проверка

In [24]:
A = np.random.rand(5,5)
f = np.random.rand(5)

In [25]:
x = gaussian(A, f)

print(A@x)
print(f)

[[0.         0.         0.         0.         0.        ]
 [0.25764382 0.6855051  0.25025911 0.95562458 0.28177488]
 [0.         0.0671283  0.89234173 0.49249202 0.04093266]
 [0.         0.         0.28085231 0.48483122 0.09753412]
 [0.         0.         0.         0.25905381 0.28700987]]
[0.         1.30280296 0.89370698 1.11226048 1.63229611]
delta norm: 2.5293322622916294
[0.87822307 0.38618883 0.73437477 0.85749108 0.52684888]
[0.87822307 0.38618883 0.73437477 0.85749108 0.52684888]
